# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{80993512634: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

80993512634

In [15]:
x.id

63853145472

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:63853145472 -> bob:80993512634]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# try this project here!
alice = sy.VirtualWorker(hook, id="alice")

In [22]:
alice._objects

{}

In [23]:
x = th.tensor([1, 2, 3, 7, 8])

In [24]:
x_ptr = x.send(bob, alice)

In [25]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:70718076011 -> bob:25230562567]
	-> (Wrapper)>[PointerTensor | me:43900594024 -> alice:13123083117]

In [26]:
x_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:70718076011 -> bob:25230562567],
 'alice': (Wrapper)>[PointerTensor | me:43900594024 -> alice:13123083117]}

In [27]:
x_ptr.get()

[tensor([1, 2, 3, 7, 8]), tensor([1, 2, 3, 7, 8])]

In [28]:
x = th.tensor([1, 1, 0, 1, 0]).send(bob, alice)
x.get(sum_results=True)

tensor([2, 2, 0, 2, 0])

# Lesson: Introducing Remote Arithmetic

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [30]:
x

(Wrapper)>[PointerTensor | me:7513863953 -> bob:42366213213]

In [31]:
y

(Wrapper)>[PointerTensor | me:59704481731 -> bob:20566784837]

In [32]:
z = x + y

In [33]:
z

(Wrapper)>[PointerTensor | me:94471448189 -> bob:76626777116]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:96133788896 -> bob:39921684574]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [38]:
z = (x + y).sum()

In [39]:
z.backward()

(Wrapper)>[PointerTensor | me:94375109558 -> bob:49080836837]

In [40]:
x = x.get()

In [41]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [42]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [43]:
# try this project here!
input  = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [44]:
weights = th.tensor([[0.], [0]], requires_grad=True).send(bob)

In [45]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [46]:
bob = bob.clear_objects()

In [47]:
bob._objects

{}

In [48]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [49]:
bob._objects

{12051590775: tensor([1, 2, 3, 4, 5])}

In [50]:
del x

In [51]:
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{61479196163: tensor([1, 2, 3, 4, 5])}

In [54]:
x = "asdf"

In [55]:
bob._objects

{}

In [56]:
Out

AttributeError: 'Tensor' object has no attribute 'child'

In [57]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [58]:
x

(Wrapper)>[PointerTensor | me:2182380922 -> bob:1021231148]

In [59]:
bob._objects

{1021231148: tensor([1, 2, 3, 4, 5])}

In [60]:
x = "asdf"

In [61]:
bob._objects

{1021231148: tensor([1, 2, 3, 4, 5])}

In [62]:
del x

In [63]:
bob._objects

{1021231148: tensor([1, 2, 3, 4, 5])}

In [64]:
bob = bob.clear_objects()
bob._objects

{}

In [65]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [66]:
bob._objects

{73727512952: tensor([1, 2, 3, 4, 5])}

In [67]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [68]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:92395567843 -> bob:37288778624]
Tensor B: tensor([1, 1, 1, 1, 1])

In [69]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [70]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [71]:
print(bob._objects)
bob = bob.clear_objects()
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)
z = x + y
bob._objects

{37288778624: tensor([1, 2, 3, 4, 5]), 59763549684: tensor([1, 2, 3, 4, 5])}


{36093168340: tensor([1, 2, 3, 4, 5]),
 39500710362: tensor([1, 1, 1, 1, 1]),
 55710248821: tensor([2, 3, 4, 5, 6])}

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [65]:
from torch import nn, optim

In [66]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [67]:
# A Toy Model
model = nn.Linear(2,1)

In [68]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [69]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(10.5832)
tensor(1.6297)
tensor(0.6997)
tensor(0.4557)
tensor(0.3180)
tensor(0.2253)
tensor(0.1609)
tensor(0.1158)
tensor(0.0839)
tensor(0.0612)
tensor(0.0449)
tensor(0.0331)
tensor(0.0245)
tensor(0.0183)
tensor(0.0136)
tensor(0.0102)
tensor(0.0077)
tensor(0.0058)
tensor(0.0044)
tensor(0.0033)


In [70]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [71]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [72]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [76]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [77]:
train()

tensor(0.8085, requires_grad=True)
tensor(0.6540, requires_grad=True)
tensor(0.2680, requires_grad=True)
tensor(0.4742, requires_grad=True)
tensor(0.1828, requires_grad=True)
tensor(0.3338, requires_grad=True)
tensor(0.1325, requires_grad=True)
tensor(0.2364, requires_grad=True)
tensor(0.0971, requires_grad=True)
tensor(0.1687, requires_grad=True)
tensor(0.0716, requires_grad=True)
tensor(0.1213, requires_grad=True)
tensor(0.0531, requires_grad=True)
tensor(0.0877, requires_grad=True)
tensor(0.0395, requires_grad=True)
tensor(0.0637, requires_grad=True)
tensor(0.0294, requires_grad=True)
tensor(0.0465, requires_grad=True)
tensor(0.0220, requires_grad=True)
tensor(0.0341, requires_grad=True)
tensor(0.0165, requires_grad=True)
tensor(0.0251, requires_grad=True)
tensor(0.0123, requires_grad=True)
tensor(0.0185, requires_grad=True)
tensor(0.0093, requires_grad=True)
tensor(0.0136, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0101, requires_grad=True)
tensor(0.0052, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [78]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [79]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [80]:
x = x.send(alice)

In [81]:
bob._objects

{17863325707: tensor([1, 2, 3, 4, 5])}

In [82]:
alice._objects

{46979563154: (Wrapper)>[PointerTensor | alice:46979563154 -> bob:17863325707]}

In [83]:
y = x + x

In [84]:
y

(Wrapper)>[PointerTensor | me:15166200114 -> alice:32189249172]

In [85]:
bob._objects

{17863325707: tensor([1, 2, 3, 4, 5]),
 57480702026: tensor([ 2,  4,  6,  8, 10])}

In [86]:
alice._objects

{46979563154: (Wrapper)>[PointerTensor | alice:46979563154 -> bob:17863325707],
 32189249172: (Wrapper)>[PointerTensor | alice:32189249172 -> bob:57480702026]}

In [87]:
jon = sy.VirtualWorker(hook, id="jon")

In [88]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [89]:
bob._objects

{22522765763: tensor([1, 2, 3, 4, 5])}

In [90]:
alice._objects

{82422774569: (Wrapper)>[PointerTensor | alice:82422774569 -> bob:22522765763]}

In [91]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:82422774569 -> bob:22522765763]

In [92]:
bob._objects

{22522765763: tensor([1, 2, 3, 4, 5])}

In [93]:
alice._objects

{}

In [94]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [95]:
bob._objects

{}

In [96]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [97]:
bob._objects

{42022236414: tensor([1, 2, 3, 4, 5])}

In [98]:
alice._objects

{2313517320: (Wrapper)>[PointerTensor | alice:2313517320 -> bob:42022236414]}

In [99]:
del x

In [100]:
bob._objects

{}

In [101]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [102]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [103]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [104]:
bob._objects

{12332796956: tensor([1, 2, 3, 4, 5])}

In [105]:
alice._objects

{}

In [106]:
x.move(alice)

(Wrapper)>[PointerTensor | me:93580224867 -> alice:93580224867]

In [107]:
bob._objects

{}

In [108]:
alice._objects

{93580224867: tensor([1, 2, 3, 4, 5])}

In [109]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [110]:
bob._objects

{18454364025: tensor([1, 2, 3, 4, 5])}

In [111]:
alice._objects

{93580224867: tensor([1, 2, 3, 4, 5]),
 97900393742: (Wrapper)>[PointerTensor | alice:97900393742 -> bob:18454364025]}

In [112]:
x.remote_get()

(Wrapper)>[PointerTensor | me:40392664660 -> alice:97900393742]

In [113]:
bob._objects

{}

In [114]:
alice._objects

{93580224867: tensor([1, 2, 3, 4, 5]), 97900393742: tensor([1, 2, 3, 4, 5])}

In [115]:
x.move(bob)

(Wrapper)>[PointerTensor | me:40392664660 -> bob:40392664660]

In [116]:
x

(Wrapper)>[PointerTensor | me:40392664660 -> bob:40392664660]

In [117]:
bob._objects

{40392664660: tensor([1, 2, 3, 4, 5])}

In [118]:
alice._objects

{93580224867: tensor([1, 2, 3, 4, 5])}